In [ ]:
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io

In [ ]:
# To run the server: uvicorn server:object_detection_server --reload

### Setup resolution

In [ ]:
# Possible image shapes for NOVIGA webcam
WIDTH, HEIGHT = [(640, 360), (640, 480), (800, 600), (1280, 720), (1280, 960)][1]
WIDTH, HEIGHT

### Detect a single image

In [ ]:
# open camera and configure it
cap = cv2.VideoCapture(0)
cap.set(3, WIDTH)
cap.set(4, HEIGHT)

In [ ]:
# capture frame
ret, bgr_frame_i = cap.read()

In [ ]:
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
# plot image with detected objects
plt.imshow(cv2.cvtColor(bgr_frame_i, cv2.COLOR_BGR2RGB))

In [ ]:
bgr_frame_i.shape

In [ ]:
# encode image as jpeg
_, img_encoded = cv2.imencode('.jpg', bgr_frame_i)

In [ ]:
img_encoded.shape

In [ ]:
# send http request with image and receive response
f = {'binary_file': img_encoded.tobytes()}
response = requests.post(r'http://127.0.0.1:8000/detect_object', files=f)
res = response.content

In [ ]:
# decode binary
array_received = np.frombuffer(res, np.uint8)
array_received.shape

In [ ]:
# reshape array
img_bgr_received = array_received.reshape(bgr_frame_i.shape)

In [ ]:
plt.imshow(cv2.cvtColor(img_bgr_received, cv2.COLOR_BGR2RGB))

### Detect a streaming video

In [ ]:
# Configure camera
cap = cv2.VideoCapture(0)
cap.set(3, WIDTH)
cap.set(4, HEIGHT)

while(True):
    # Capture frame-by-frame
    ret, bgr_frame_i = cap.read()
    
    # encode image as jpeg
    _, img_encoded = cv2.imencode('.jpg', bgr_frame_i)
    
    # send http request with image and receive response
    f = {'binary_file': img_encoded.tobytes()}
    #response = requests.post(r'http://127.0.0.1:8000/detect_object', files=f)
    response = requests.post(r'http://127.0.0.1:8000/mirrow', files=f)
    res = response.content

    # decode binary
    array_received = np.frombuffer(res, np.uint8)

    # reshape array
    img_bgr_received = array_received.reshape(bgr_frame_i.shape)

    cv2.imshow('Received image', img_bgr_received)
    cv2.imshow('Sent Image', bgr_frame_i)

    #Waits for a user input to quit the application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


print('We reached the end...')

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

### one way (Send image, perform object detection on server side without sending image back)

In [ ]:
# Configure camera
cap = cv2.VideoCapture(0)
cap.set(3, WIDTH)
cap.set(4, HEIGHT)

while(True):
    # Capture frame-by-frame
    ret, bgr_frame_i = cap.read()
    
    # encode image as jpeg
    _, img_encoded = cv2.imencode('.jpg', bgr_frame_i)
    
    # send http request with image and receive response
    f = {'binary_file': img_encoded.tobytes()}
    #response = requests.post(r'http://127.0.0.1:8000/detect_object', files=f)
    response = requests.post(r'http://127.0.0.1:8000/one_way', files=f)
    res = response.content

    # decode binary
    print(res)

    cv2.imshow('Sent Image', bgr_frame_i)

    #Waits for a user input to quit the application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


print('We reached the end...')

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()